In [ ]:
!git clone https://github.com/AI4Finance-Foundation/FinGPT.git
%cd FinGPT



Cloning into 'FinGPT'...
remote: Enumerating objects: 2743, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 2743 (delta 33), reused 18 (delta 18), pack-reused 2691 (from 4)
Receiving objects: 100% (2743/2743), 12.02 MiB | 15.86 MiB/s, done.
Resolving deltas: 100% (1614/1614), done.
/content/FinGPT


In [ ]:
!pip install -r requirements.txt
!pip install -q transformers datasets sentencepiece



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 

In [ ]:
from transformers import pipeline

sentiment_model = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
from os import cpu_count
print(cpu_count())

2


In [ ]:
# Example financial news headline
text = "Apple shares fall after iPhone sales miss expectations"
result = sentiment_model(text)
print(result)


[{'label': 'Negative', 'score': 0.9999992847442627}]


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataframe = pd.read_csv("/content/data.csv")

dataframe.head()


,Datetime,News,Link
0,2024-01-01,disney marathon how to register and qualify ch...,https://economictimes.indiatimes.com/news/inte...
1,2024-01-01,bharti airtel services to acquire 97 1 stake i...,https://economictimes.indiatimes.com/industry/...
2,2024-01-01,fallout 76 nuke codes launch site locations he...,https://economictimes.indiatimes.com/news/inte...
3,2024-01-01,is retirement on your list in 2024 here are de...,https://economictimes.indiatimes.com/news/inte...
4,2024-01-01,raise a glass to gift city why the recent amen...,https://economictimes.indiatimes.com/opinion/e...


In [ ]:
!pip install colorama

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from colorama import Fore, Style, init
import pandas as pd
import os
import time
import warnings
from transformers import pipeline

init(autoreset=True)

# Global Constants
INPUT_FILE = "/content/data.csv"
OUTPUT_FILE = "/content/sentiment_data.csv"
NUM_PROCESSES = min(4, os.cpu_count() - 1)
MODEL = "yiyanghkust/finbert-tone"

def load_model(model):
  sentiment_model = pipeline("sentiment-analysis", model=model)
  return sentiment_model

sentiment_model = load_model(MODEL)




def find_sentiment(text):
  try:
    print(f"{Fore.GREEN} Input:- {text[:20]}{Style.RESET_ALL}")
    result = sentiment_model(text)
    print(f"{Fore.GREEN} Output:- {result}{Style.RESET_ALL}")
    return list((text, result[0]["label"], result[0]["score"]))
  except Exception as e:
    print(f"{Fore.RED} Error: {e}{Style.RESET_ALL}")
    return list((text, np.nan, np.nan))

def process_texts_in_parallel(texts):
    """Process multiple links in parallel using threads."""
    print(f"{Fore.CYAN}🔹 Using {NUM_PROCESSES} parallel threads...{Style.RESET_ALL}")

    with ThreadPoolExecutor(NUM_PROCESSES) as executor:
        results = list(executor.map(find_sentiment, texts))

    return results


def sentiment_analysis_of_text_data():
    """Main function to scrape news articles from links in data.csv."""
    if not os.path.exists(INPUT_FILE):
        print(f"{Fore.RED}❌ Error: '{INPUT_FILE}' not found!{Style.RESET_ALL}")
        return

    df = pd.read_csv(INPUT_FILE)

    if "News" not in df.columns:
        print(f"{Fore.RED}❌ Error: 'News' column missing in CSV!{Style.RESET_ALL}")
        return

    texts = df["News"].dropna().tolist()

    # when the scrapping stops unexpectedly
    already_processed_texts = set()
    if os.path.exists(OUTPUT_FILE):
        already_processed_texts = set(
            pd.read_csv(OUTPUT_FILE)["News"].dropna().tolist()
        )

    # filter the links if they are already processed

    links = [text for text in texts if text not in already_processed_texts]

    # Split into batches to avoid excessive memory usage
    batch_size = 500
    total_batches = (len(texts) // batch_size) + 1

    all_results = []

    for i in range(total_batches):
        batch_texts = texts[i * batch_size: (i + 1) * batch_size]
        print(f"{Fore.YELLOW}📌 Processing batch {i + 1}/{total_batches} ({len(batch_texts)} links){Style.RESET_ALL}")

        results = process_texts_in_parallel(batch_texts)  # Removed `[0]`
        all_results.extend(results)

        # Save intermediate results
        temp_df = pd.DataFrame(all_results, columns=["News", "sentiment_label", "sentiment_score"])
        temp_df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")

    print(f"{Fore.GREEN}✔️ Sentiment Analysis complete! Data saved to '{OUTPUT_FILE}'{Style.RESET_ALL}")

# Run the scraper
# if __name__ == "__main__":
#    scrape_news()


Device set to use cpu


In [ ]:
if __name__ == "__main__":
   sentiment_analysis_of_text_data()

Streaming output truncated to the last 5000 lines.
 Input:- christmas 2023 resta
 Output:- [{'label': 'Neutral', 'score': 0.9999969005584717}]
 Input:- workers from all ove
 Output:- [{'label': 'Neutral', 'score': 0.9985222220420837}]
 Input:- india and france pro
 Output:- [{'label': 'Neutral', 'score': 0.934623122215271}]
 Input:- slaying the nostalgi
 Output:- [{'label': 'Neutral', 'score': 0.9996726512908936}]
 Input:- top 2023 hollywood h
 Output:- [{'label': 'Neutral', 'score': 0.9998931884765625}]
 Input:- manipur move to resu
 Output:- [{'label': 'Negative', 'score': 0.9999879598617554}]
 Input:- woman gives birth tw
 Output:- [{'label': 'Neutral', 'score': 0.9996504783630371}]
 Input:- in line with gandhis
 Output:- [{'label': 'Neutral', 'score': 0.9994848966598511}]
 Input:- bad behaviour in mov
 Output:- [{'label': 'Negative', 'score': 0.9067023396492004}]
 Input:- mumbai indians denie
 Output:- [{'label': 'Neutral', 'score': 0.9993564486503601}]
 Input:- cci clears alpha al